In [12]:
import pandas as pd
import json
import re

# --- Configuration ---
file_path = 'DUOLIFE Ingredients.xlsx'

# --- Load all sheets from the Excel file ---
try:
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names
    sanitized_sheet_names = {sheet: sheet.replace('.csv', '').strip() for sheet in sheet_names}
    data_frames = {sanitized_sheet_names[sheet]: pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names}
    print("Successfully loaded sheets.")
except Exception as e:
    print(f"An error occurred: {e}")

def clean_text(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'\s+', ' ', text).strip()

def sanitize_for_id(name: str) -> str:
    """
    Creates a consistent, safe, and NORMALIZED ID from a name string.
    This is the key fix to merge similar ingredients.
    """
    # Convert to lowercase
    text = name.lower()
    
    # Remove common descriptive words that cause fragmentation
    words_to_remove = ['extract', 'herb', 'leaf', 'fruit', 'juice', 'seed', 'root', 'oil', 'powder']
    for word in words_to_remove:
        text = text.replace(word, '')
        
    # Remove content in parentheses
    text = re.sub(r'\(.*\)', '', text)
    
    # Sanitize the remaining string
    sanitized_name = re.sub(r'[^a-z0-9_]+', '_', text).strip('_')
    return f"ingredient_{sanitized_name}"

Successfully loaded sheets.


In [13]:
all_nodes = []
node_ids = set()
ingredient_name_to_id_map = {} # The robust lookup map

# --- Step 2.1: Extract and Unify Ingredient Nodes FIRST ---
df_ingredients1 = data_frames.get('Ingredients')
df_ingredients2 = data_frames.get('Ingredients v2')
unique_ingredient_names = set()

if df_ingredients1 is not None:
    for _, row in df_ingredients1.iterrows():
        unique_ingredient_names.add(clean_text(row.get('Ingredient Name')))
if df_ingredients2 is not None:
    for _, row in df_ingredients2.iterrows():
        unique_ingredient_names.add(clean_text(row.get('Ingredient Name')))

for name in sorted(list(unique_ingredient_names)): # sorted for consistent IDs
    if name:
        ing_id = sanitize_for_id(name)
        if ing_id not in node_ids:
            all_nodes.append({
                "id": ing_id,
                "type": "Ingredient",
                "name": name
            })
            node_ids.add(ing_id)
            # Add the cleaned, lowercased name and its ID to our lookup map
            ingredient_name_to_id_map[name.lower()] = ing_id

# --- Step 2.2: Extract Product Nodes ---
df_products = data_frames.get('Products')
if df_products is not None:
    for _, row in df_products.iterrows():
        product_id = f"product_{row.get('Product ID')}"
        if product_id not in node_ids and pd.notna(row.get('Product Name')):
            all_nodes.append({
                "id": product_id,
                "type": "Product",
                "name": clean_text(row.get('Product Name')),
                "category": clean_text(row.get('Category')),
                "link": clean_text(row.get('Link'))
            })
            node_ids.add(product_id)

# --- Step 2.3: Extract Cosmetic Nodes ---
df_cosmetics = data_frames.get('Cosmetics')
if df_cosmetics is not None:
    for _, row in df_cosmetics.iterrows():
        cosmetic_id = f"cosmetic_{row.get('Product ID')}"
        if cosmetic_id not in node_ids and pd.notna(row.get('Product Name')):
            all_nodes.append({
                "id": cosmetic_id,
                "type": "Cosmetic",
                "name": clean_text(row.get('Product Name')),
                "product_type": clean_text(row.get('Type'))
            })
            node_ids.add(cosmetic_id)

print(f"Extracted a total of {len(all_nodes)} unique nodes.")
print(f"Created an ingredient lookup map with {len(ingredient_name_to_id_map)} entries.")

Extracted a total of 317 unique nodes.
Created an ingredient lookup map with 240 entries.


In [14]:
all_edges = []

# --- Create Product -> Ingredient Edges ---
if df_products is not None:
    for _, row in df_products.iterrows():
        if pd.isna(row.get('Product ID')) or pd.isna(row.get('Ingredient Name')):
            continue
        source_id = f"product_{row.get('Product ID')}"
        # The 'Ingredient Name' column contains comma-separated NAMES
        ingredient_names_str = str(row.get('Ingredient Name', ''))
        ingredient_names_list = [clean_text(name) for name in ingredient_names_str.split(',') if clean_text(name)]
        
        for ing_name in ingredient_names_list:
            # Use the lookup map to find the correct ID
            target_id = ingredient_name_to_id_map.get(ing_name.lower())
            if source_id and target_id:
                all_edges.append({
                    "source": source_id,
                    "target": target_id,
                    "label": "CONTAINS"
                })

# --- Create Cosmetic -> Ingredient Edges ---
if df_cosmetics is not None:
    df_cosmetics.columns = [col.strip() for col in df_cosmetics.columns]
    for _, row in df_cosmetics.iterrows():
        if pd.isna(row.get('Product ID')):
            continue
        source_id = f"cosmetic_{row.get('Product ID')}"
        
        for col_name in df_cosmetics.columns:
            # The scattered 'Ingredient IDs' columns contain NAMES
            if 'Ingredient IDs' in col_name and pd.notna(row[col_name]):
                ing_name = clean_text(row[col_name])
                # Use the lookup map to find the correct ID
                target_id = ingredient_name_to_id_map.get(ing_name.lower())
                if source_id and target_id:
                    all_edges.append({
                        "source": source_id,
                        "target": target_id,
                        "label": "CONTAINS"
                    })

print(f"Extracted a total of {len(all_edges)} edges (relationships) using the lookup map.")

Extracted a total of 271 edges (relationships) using the lookup map.


In [15]:
# --- Print sample nodes and edges for verification ---
print("\n=== SAMPLE NODES ===")
print(f"Total nodes: {len(all_nodes)}")
print("\nFirst 5 nodes:")
for i, node in enumerate(all_nodes[:5]):
    print(f"{i+1}. {node}")

print("\nSample nodes by type:")
ingredient_nodes = [node for node in all_nodes if node['type'] == 'Ingredient']
product_nodes = [node for node in all_nodes if node['type'] == 'Product']
cosmetic_nodes = [node for node in all_nodes if node['type'] == 'Cosmetic']

print(f"\nIngredient nodes ({len(ingredient_nodes)} total):")
for node in ingredient_nodes[:3]:
    print(f"  {node}")

print(f"\nProduct nodes ({len(product_nodes)} total):")
for node in product_nodes[:3]:
    print(f"  {node}")

print(f"\nCosmetic nodes ({len(cosmetic_nodes)} total):")
for node in cosmetic_nodes[:3]:
    print(f"  {node}")

print("\n=== SAMPLE EDGES ===")
print(f"Total edges: {len(all_edges)}")
print("\nFirst 10 edges:")
for i, edge in enumerate(all_edges[:10]):
    print(f"{i+1}. {edge}")

print("\nEdges by source type:")
product_edges = [edge for edge in all_edges if edge['source'].startswith('product_')]
cosmetic_edges = [edge for edge in all_edges if edge['source'].startswith('cosmetic_')]

print(f"\nProduct -> Ingredient edges ({len(product_edges)} total):")
for edge in product_edges[:5]:
    print(f"  {edge}")

print(f"\nCosmetic -> Ingredient edges ({len(cosmetic_edges)} total):")
for edge in cosmetic_edges[:5]:
    print(f"  {edge}")

print("\n=== INGREDIENT LOOKUP MAP SAMPLE ===")
print(f"Total lookup entries: {len(ingredient_name_to_id_map)}")
print("First 10 lookup map entries:")
for i, (name, id_val) in enumerate(list(ingredient_name_to_id_map.items())[:10]):
    print(f"{i+1}. '{name}' -> '{id_val}'")


=== SAMPLE NODES ===
Total nodes: 317

First 5 nodes:
1. {'id': 'ingredient_astragin', 'type': 'Ingredient', 'name': 'ASTRAGIN®'}
2. {'id': 'ingredient_acacia_fiber', 'type': 'Ingredient', 'name': 'Acacia (Acacia senegal) fiber'}
3. {'id': 'ingredient_acai_berry', 'type': 'Ingredient', 'name': 'Acai (Euterpe Badiocarpa) berry extract'}
4. {'id': 'ingredient_acai', 'type': 'Ingredient', 'name': 'Acai (Euterpe oleracea) fruit extract'}
5. {'id': 'ingredient_acerola', 'type': 'Ingredient', 'name': 'Acerola (Malpighia glabra) fruit extract'}

Sample nodes by type:

Ingredient nodes (240 total):
  {'id': 'ingredient_astragin', 'type': 'Ingredient', 'name': 'ASTRAGIN®'}
  {'id': 'ingredient_acacia_fiber', 'type': 'Ingredient', 'name': 'Acacia (Acacia senegal) fiber'}
  {'id': 'ingredient_acai_berry', 'type': 'Ingredient', 'name': 'Acai (Euterpe Badiocarpa) berry extract'}

Product nodes (57 total):
  {'id': 'product_D1', 'type': 'Product', 'name': 'DuoLife Day and Night', 'category': 'Diata

In [16]:
# --- Save nodes and edges to JSON files ---
with open('nodes.json', 'w', encoding='utf-8') as f:
    json.dump(all_nodes, f, indent=4)

with open('edges.json', 'w', encoding='utf-8') as f:
    json.dump(all_edges, f, indent=4)

print("Successfully saved corrected 'nodes.json' and 'edges.json'.")

Successfully saved corrected 'nodes.json' and 'edges.json'.
